<a href="https://colab.research.google.com/github/Im-LAKSH/OCT_Image_Classification/blob/main/Normal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import os
import random
import cv2
from PIL import Image
import glob
from torch.utils.data import Dataset, DataLoader
from google.colab import drive


drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")


class DCGANConfig:

    dataset_path = '/content/drive/MyDrive/PBL/OCTID/NORMAL'
    output_folder = '/content/drive/MyDrive/Syn_Dataset/NORMAL'


    image_size = 128
    nc = 1
    nz = 100
    ngf = 64
    ndf = 64


    num_epochs = 200
    batch_size = 16
    lr = 0.0002
    beta1 = 0.5

config = DCGANConfig()
os.makedirs(config.output_folder, exist_ok=True)


class OCTDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        for ext in ['*.png', '*.jpg', '*.jpeg', '*.PNG']:
            self.image_paths.extend(glob.glob(os.path.join(root_dir, ext)))
        print(f"📂 Found {len(self.image_paths)} images in {root_dir}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            img_path = self.image_paths[idx]
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is None: image = np.array(Image.open(img_path).convert('L'))
            image = Image.fromarray(image)
            if self.transform: image = self.transform(image)
            return image
        except: return torch.zeros(1, config.image_size, config.image_size)

transform = transforms.Compose([
    transforms.Resize((config.image_size, config.image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = OCTDataset(config.dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True, drop_last=True)


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1: nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1: nn.init.normal_(m.weight.data, 1.0, 0.02)
    elif classname.find('constant') != -1: nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(config.nz, config.ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(config.ngf * 8), nn.ReLU(True),
            nn.ConvTranspose2d(config.ngf * 8, config.ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(config.ngf * 4), nn.ReLU(True),
            nn.ConvTranspose2d(config.ngf * 4, config.ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(config.ngf * 2), nn.ReLU(True),
            nn.ConvTranspose2d(config.ngf * 2, config.ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(config.ngf), nn.ReLU(True),
            nn.ConvTranspose2d(config.ngf, config.ngf // 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(config.ngf // 2), nn.ReLU(True),
            nn.ConvTranspose2d(config.ngf // 2, config.nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, input): return self.main(input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(config.nc, config.ndf // 2, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(config.ndf // 2, config.ndf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(config.ndf), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(config.ndf, config.ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(config.ndf * 2), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(config.ndf * 2, config.ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(config.ndf * 4), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(config.ndf * 4, config.ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(config.ndf * 8), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(config.ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, input): return self.main(input).view(-1, 1).squeeze(1)

netG = Generator().to(device)
netD = Discriminator().to(device)
netG.apply(weights_init)
netD.apply(weights_init)


criterion = nn.BCELoss()

optimizerD = optim.Adam(netD.parameters(), lr=0.0001, betas=(config.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(config.beta1, 0.999))

print(f"🚀 Starting Stabilized Training for {config.num_epochs} epochs...")

for epoch in range(config.num_epochs):
    for i, data in enumerate(dataloader, 0):

        netD.zero_grad()
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)


        label = torch.full((b_size,), 0.9, dtype=torch.float, device=device)


        noise_amp = 0.1 * (1 - epoch/config.num_epochs)
        real_noisy = real_cpu + (torch.randn_like(real_cpu) * noise_amp)

        output = netD(real_noisy)
        errD_real = criterion(output, label)
        errD_real.backward()


        noise = torch.randn(b_size, config.nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(0.0)


        fake_noisy = fake + (torch.randn_like(fake) * noise_amp)

        output = netD(fake_noisy.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        optimizerD.step()


        netG.zero_grad()
        label.fill_(1.0)
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        optimizerG.step()

    if epoch % 20 == 0:
        print(f'[{epoch}/{config.num_epochs}] Loss_D: {errD_real.item()+errD_fake.item():.4f} Loss_G: {errG.item():.4f}')


print(f"💾 Generating 1000 images to: {config.output_folder}")
netG.eval()
with torch.no_grad():
    for i in range(1000):
        noise = torch.randn(1, config.nz, 1, 1, device=device)
        fake = netG(noise)


        fake = (fake + 1) / 2.0

        save_path = os.path.join(config.output_folder, f"syn_{i:04d}.png")
        vutils.save_image(fake, save_path)

print("✅ DONE! Images saved to Google Drive.")

KeyboardInterrupt: 